In [1]:
#pip install cookiecutter

# Solution Planning

In [2]:
# Input

1. Business Problem

    Selecting the most valuable customers to join a fidelization program

2. Data

    E-commerce sales during a year

In [3]:
# Output

1. List of customer will be part of insider program

    client_id  |  is_insider
     10323     |  yes

In [4]:
# Task

1. Quem são as pessoas elegíveis para participar do programa de Insiders ?
    - O que é ser elegivel? O que são clientes de maior 'valor'?
    - Faturamento:
        - Alto Ticket Médio
        - Alto LTV
        - Baixa recência
        - Alto basket size
        - Baixa probabilidade de churn
        - Alta previsão de LTV
        - Alta propensão de compra
        
    - Custo:
        - Baixa taxa de devolução
        
    - Experiencia de compra:
        - Média alta das avaliações
        
2. Quantos clientes farão parte do grupo?
    - Número total de clientes
    - % do grupo insiders
    
3. Quais as principais características desses clientes ?
    - Escrever características dos clientes:
        - Idade
        - Localização
        
    - Escrever características do consumo
        - Atributos de clusterizaçao

4. Qual a porcentagem de contribuição do faturamento, vinda do Insiders ?
    - Faturamento total do ano
    - Faturamento do grupo Insiders
    
    
5. Qual a expectativa de faturamento desse grupo para os próximos meses ?
    - LTV do grupo Insiders
    - Análise Cohort
    
6. Quais as condições para uma pessoa ser elegível ao Insiders ?
    - Definir a periodicidade (1 mes, 3 meses)
    - A pessoa precisa ser similar ou parecido com uma pessoa grupo
        
7. Quais as condições para uma pessoa ser removida do Insiders ?
    - Definir a periodicidade (1 mes, 3 meses)
    - A pessoa nao é ser similar ou parecido com uma pessoa grupo
    
8. Qual a garantia que o programa Insiders é melhor que o restante da base ?
    - Teste A/B
    - Teste A/B Bayesiano
    - Teste de hipóteses
    
9. Quais ações o time de marketing pode realizar para aumentar o faturamento?
    - Desconto
    - Preferencia de compra
    - Visita a empresa

Benchmark de soluções

# Imports

In [25]:
import numpy as np
import pandas as pd
import seaborn as sns

from matplotlib import pyplot as plt

# Helper Functions

# Data Import

In [26]:
#load data
df = pd.read_csv('Ecommerce.csv',encoding="ISO-8859-1")

#drop
df = df.drop(columns=['Unnamed: 8'], axis=1)

# 1.0 Data Description

In [27]:
df1 = df.copy()

## 1.1. Rename Columns 

In [28]:
cols_new = df1.columns

cols_new = ['invoice_no', 'stock_code', 'description', 'quantity', 'invoice_date',
       'unit_price', 'customer_id', 'country']

df1.columns = cols_new

## 1.2. Data Dimensions

In [29]:
print('Number of columns: ', df1.shape[1])
print('Number of rows: ', df1.shape[0])

Number of columns:  8
Number of rows:  541909


## 1.3. Data Type

In [30]:
df1.dtypes

invoice_no       object
stock_code       object
description      object
quantity          int64
invoice_date     object
unit_price      float64
customer_id     float64
country          object
dtype: object

## 1.4. NA Check

In [31]:
df1.isna().sum()

invoice_no           0
stock_code           0
description       1454
quantity             0
invoice_date         0
unit_price           0
customer_id     135080
country              0
dtype: int64

## 1.5. Replace NA

In [32]:
df1 = df1.dropna(subset=['description', 'customer_id'])
print('% dataset removed: {:.2f}'.format(1 - (df1.shape[0]/df.shape[0])))

% dataset removed: 0.25


## 1.6. Change Types

In [33]:
#invoice data
df1['invoice_date'] = pd.to_datetime(df1['invoice_date'], format= '%d-%b-%y')

#customer id
df1['customer_id'] = df1['customer_id'].astype(int)

## 1.7. Descriptive Statistics

# 2.0 Feature Engineering

In [34]:
df2 = df1.copy()

## 2.1. Feature Creation

In [37]:
df_ref = df2.drop(['invoice_no', 'stock_code', 'description', 'quantity',
              'invoice_date', 'unit_price', 'country'], axis=1).drop_duplicates(ignore_index=True)

In [38]:
df_ref.head()

,customer_id
0,17850
1,13047
2,12583
3,13748
4,15100


In [39]:
#gross revenue
df2['gross_revenue'] = df2['quantity']*df2['unit_price']

#monetary
df_monetary = df2[['customer_id', 'gross_revenue']].groupby('customer_id').sum().reset_index()
df_ref = pd.merge(df_ref, df_monetary, on='customer_id', how='left')

#recency
df_recency = df2[['customer_id', 'invoice_date']].groupby('customer_id').max().reset_index()
df_recency['recency_days'] = (df2['invoice_date'].max() - df_recency['invoice_date']).dt.days
df_recency = df_recency[['customer_id', 'recency_days']].copy()
df_ref = pd.merge(df_ref, df_recency, on='customer_id', how='left')

#frequency
df_freq = df2[['customer_id','invoice_no']].drop_duplicates().groupby('customer_id').count().reset_index()
df_ref = pd.merge(df_ref, df_freq, on='customer_id', how='left')